### Data Description:

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

Attributes:
id : unique identifier for candidate (numeric)
job_title : job title for candidate (text)
location : geographical location for candidate (text)
connections: number of connections candidate has, 500+ means over 500 (text)
Output (desired target):
fit - how fit the candidate is for the role? (numeric, probability between 0-1)
Keywords: “Aspiring human resources” or “seeking human resources”

### Project description:
Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(".\potential-talents - Aspiring human resources - seeking human resources.csv")

In [4]:
df.head(40)

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
6,7,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
7,8,HR Senior Specialist,San Francisco Bay Area,500+,NaN
8,9,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
9,10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [6]:
df.describe()

,id,fit
count,104.000000,0.0
mean,52.500000,NaN
std,30.166206,NaN
min,1.000000,NaN
25%,26.750000,NaN
50%,52.500000,NaN
75%,78.250000,NaN
max,104.000000,NaN


In [7]:
import nltk

In [13]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    | 

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat

[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_tagset to
[nltk_data]    |     C:\Users\alima\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_tagset is already up-to-date!
[nltk_data]    | Downloading package universal

True

In [8]:
df.isnull().sum()

id              0
job_title       0
location        0
connection      0
fit           104
dtype: int64

In [9]:
df = df.drop('fit', axis=1)

In [10]:
df['job_title'] = df['job_title'].astype(str).str.lower()
df['locaiton'] = df['location'].astype(str).str.lower()

In [11]:
#### to check if there are any empty string with only spaces! 
df['job_title'].str.isspace().sum()

0

In [12]:
df['location'].str.isspace().sum()

0

In [13]:
df['job_title'].value_counts()

2019 c.t. bauer college of business graduate (magna cum laude) and aspiring human resources professional                 7
aspiring human resources professional                                                                                    7
student at humber college and aspiring human resources generalist                                                        7
people development coordinator at ryan                                                                                   6
native english teacher at epik (english program in korea)                                                                5
aspiring human resources specialist                                                                                      5
hr senior specialist                                                                                                     5
student at chapman university                                                                                            4
svp, chro, marke

In [14]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [15]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [16]:
# Removing Punctuation
df['job_title'] = df['job_title'].apply(lambda text: ''.join([char for char in text if char not in string.punctuation]))

In [17]:
# Tokenizing!
df['preprocessed_job_titles'] = df['job_title'].apply(lambda title: word_tokenize(title)) 


In [18]:
# stop words!
stop_words = set(stopwords.words('english'))
remove_stopwords = lambda tokens: [token for token in tokens if token.lower() not in stop_words]
df['preprocessed_job_titles'] = df['preprocessed_job_titles'].apply(remove_stopwords)


In [19]:
#stemming!
stemmer = PorterStemmer()
df['preprocessed_job_titles'] = df['preprocessed_job_titles'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

In [20]:
# joining the tokens!
df['preprocessed_job_titles'] = df['preprocessed_job_titles'].apply(lambda title: ' '.join(title))

In [21]:
df['preprocessed_job_titles']

0      2019 ct bauer colleg busi graduat magna cum la...
1       nativ english teacher epik english program korea
2                         aspir human resourc profession
3                             peopl develop coordin ryan
4              advisori board member celal bayar univers
                             ...                        
99     aspir human resourc manag graduat may 2020 see...
100                     human resourc generalist loparex
101                          busi intellig analyt travel
102                                    alway set success
103                         director administr excel log
Name: preprocessed_job_titles, Length: 104, dtype: object

In [22]:
reference_job_title = [df['preprocessed_job_titles'][6]]
reference_job_title

['student humber colleg aspir human resourc generalist']

In [23]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['preprocessed_job_titles'])
tfidf_reference_title = vectorizer.transform(reference_job_title)
cosine_similarities = cosine_similarity(tfidf_reference_title, tfidf_matrix).flatten()


In [24]:
df['similarity_score'] = cosine_similarities
df_sorted = df.sort_values(by='similarity_score', ascending=False)

In [25]:
df_sorted.head(20)


,id,job_title,location,connection,locaiton,preprocessed_job_titles,similarity_score
49,50,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
51,52,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
36,37,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
6,7,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
8,9,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
38,39,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
24,25,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
97,98,student,"Houston, Texas Area",4,"houston, texas area",student,0.411275
28,29,aspiring human resources management student se...,"Houston, Texas Area",500+,"houston, texas area",aspir human resourc manag student seek internship,0.337110
26,27,aspiring human resources management student se...,"Houston, Texas Area",500+,"houston, texas area",aspir human resourc manag student seek internship,0.337110


## Using Spacy

In [32]:
!python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [26]:
import spacy
nlp = spacy.load("en_core_web_md")

In [27]:
# Removing punctuation
def preprocess_text(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

In [28]:
df['job_title'] = df['job_title'].apply(preprocess_text)


In [29]:
reference_text = df["job_title"][6]

In [30]:
def calculate_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    vec1 = doc1.vector.reshape(1, -1)
    vec2 = doc2.vector.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

In [31]:
df['similarity_score'] = df['job_title'].apply(lambda x: calculate_similarity(x, reference_text))

In [32]:
df = df.sort_values(by='similarity_score', ascending=False).reset_index(drop=True)


In [33]:
df

,id,job_title,location,connection,locaiton,preprocessed_job_titles,similarity_score
0,25,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
1,37,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
2,7,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
3,39,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
4,9,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
...,...,...,...,...,...,...,...
99,12,svp chro marketing communications csr officer...,"Houston, Texas Area",500+,"houston, texas area",svp chro market commun csr offic engi houston ...,0.615281
100,55,svp chro marketing communications csr officer...,"Houston, Texas Area",500+,"houston, texas area",svp chro market commun csr offic engi houston ...,0.615281
101,64,svp chro marketing communications csr officer...,"Houston, Texas Area",500+,"houston, texas area",svp chro market commun csr offic engi houston ...,0.615281
102,85,rrp brand portfolio executive at jti japan tob...,Greater Philadelphia Area,500+,greater philadelphia area,rrp brand portfolio execut jti japan tobacco i...,0.610477


## Transformers Embeddings

In [2]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.Collecting transformers<5.0.0,>=4.6.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\alima\\anaconda3\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.




  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=eac945e22747228a66ca6956be9e33a03630ed461b94ec9ef96888923d756483
  Stored in directory: c:\users\alima\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [35]:
from sentence_transformers import SentenceTransformer, util

In [36]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [37]:
embeddings = model.encode(df['job_title'])


In [43]:
def transformers_similarity(text1, text2):
    emb1 = model.encode(text1).reshape(1,-1)
    emb2 = model.encode(text2).reshape(1,-1)
    return cosine_similarity(emb1, emb2)[0][0]


In [44]:
# similarity_score = util.pytorch_cos_sim(embeddings, embeddings[6])
reference_text = df["job_title"][6]

In [45]:
df['similarity_score'] = df['job_title'].apply(lambda x: transformers_similarity(x, reference_text))

In [46]:
df = df.sort_values(by='similarity_score', ascending=False).reset_index(drop=True)

In [47]:
df

,id,job_title,location,connection,locaiton,preprocessed_job_titles,similarity_score
0,25,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
1,7,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
2,39,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
3,9,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
4,50,student at humber college and aspiring human r...,Kanada,61,kanada,student humber colleg aspir human resourc gene...,1.000000
...,...,...,...,...,...,...,...
99,77,human resources\nconflict management\npolicies...,Dallas/Fort Worth Area,409,dallas/fort worth area,human resourc conflict manag polici procedures...,0.446264
100,103,always set them up for success,Greater Los Angeles Area,500+,greater los angeles area,alway set success,0.422414
101,85,rrp brand portfolio executive at jti japan tob...,Greater Philadelphia Area,500+,greater philadelphia area,rrp brand portfolio execut jti japan tobacco i...,0.403827
102,93,admissions representative at community medical...,"Long Beach, California",9,"long beach, california",admiss repres commun medic center long beach,0.392872
